In [2]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

__Cascade Classifiers 로드__

In [21]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontface.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
glass_cascade = cv2.CascadeClassifier('haarcascade_eye_tree_eyeglasses.xml')
eyesplit_cascade = cv2.CascadeClassifier('haarcascade_lefteye_2splits.xml')

__advanced_eye_detect 함수 정의__

In [22]:
def advanced_eye_detect(img,cascade,info):
    
    ret_val=[]
    
    img_gray =  cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(img_gray)
    
    if len(faces)!=1:
        return ret_val
    
    for (x,y,w,h) in faces:
        roi_gray = img_gray[y:y+h, x:x+w]
        eyes =cascade.detectMultiScale(roi_gray,1.1,4)
    
    if not (len(eyes)==2):    
        if info == 'normal':
            return advanced_eye_detect(img,eyesplit_cascade,'split')
        elif info == 'split':
            return advanced_eye_detect(img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
        
    ret_val = eyes
    return ret_val

__get_rotated_image 함수 정의__

In [13]:
def get_rotated_image(img,eyes):
    
    eye_1 , eye_2 = eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
        
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center

    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    if not delta_x or delta_y:
        return img
    
    angle=np.arctan(delta_y/delta_x)
    angle = (angle * 180) / np.pi
    
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, (angle), 1.0)
    rotated_img = cv2.warpAffine(img, M, (w, h))
  
    return rotated_img

__calculate_rotated_eyes 함수 정의__

In [10]:
def calculate_rotated_eyes(rotated_img,cascade,info):

    ret_val=[]
    
    rotated_gray = cv2.cvtColor(rotated_img, cv2.COLOR_BGR2GRAY)
    
    faces = face_cascade.detectMultiScale(rotated_gray, 1.1, 4)
    
    if len(faces)!=1:
        return ret_val
    
    for (x,y,w,h) in faces:

        black=np.zeros(rotated_gray.shape,dtype='uint8')
        black[y:y+h, x:x+w]=rotated_gray[y:y+h, x:x+w]

        rotated_eyes =cascade.detectMultiScale(black,1.1,4)    
    
    if len(rotated_eyes)!=2:    
        if info == 'normal':
            return calculate_rotated_eyes(rotated_img,glass_cascade,'glasses')
        elif info == 'glasses':
            return ret_val
    
    ret_val = rotated_eyes
    
    return ret_val

__extract_facial_mask_area 함수 정의__

In [7]:
def extract_facial_mask_area(rotated_img,rotated_eyes):
    
    eye_1 , eye_2 = rotated_eyes

    if eye_1[0] < eye_2[0]:
        left_eye = eye_1
        right_eye = eye_2
    else:
        left_eye = eye_2
        right_eye = eye_1
    
    left_eye_center = (int(left_eye[0] + (left_eye[2] / 2)), int(left_eye[1] + (left_eye[3] / 2)))
    right_eye_center = (int(right_eye[0] + (right_eye[2]/2)), int(right_eye[1] + (right_eye[3]/2)))
    
    left_eye_x , left_eye_y = left_eye_center 
    right_eye_x , right_eye_y = right_eye_center
    
    delta_x = right_eye_x - left_eye_x
    delta_y = right_eye_y - left_eye_y
    
    L = np.sqrt(delta_x**2 + delta_y**2)
    xpad_L , xpad_R = int(0.6*L) , int(1.6*L)
    ypad_U , ypad_D = int(0.6*L) , int(1.8*L)
    
    ROI = rotated_img[left_eye_y-ypad_U:left_eye_y+ypad_D,left_eye_x-xpad_L:left_eye_x+xpad_R]
    ROI_resized = cv2.resize(ROI,(120,140))
    mask_area = ROI_resized[50:140,0:120]
    
    return mask_area

__메인 코드__

In [29]:
import os

img_path = "D:/downloads/gen_img/nose_mask_1000/" # replace it with your path

files = os.listdir(img_path)
image_files = [file for file in files if file.endswith(".png")]
mask_area=[]
for file in image_files:
    
    image=cv2.imread(img_path+file)
    print(f'{file} detection result')
    
    if image.shape[0]<400 or image.shape[1]<400 :
        w,h,c = image.shape
        image = cv2.resize(image,(h*2,w*2))
    
    eyes = advanced_eye_detect(image,eye_cascade,'normal')
    
    if len(eyes)!=2:
        print('Initial Detection Failed : not human or invalid image')
        print('='*65)
        continue
    
    print('Initial Detection success')
    rotated_image = get_rotated_image(image,eyes)
    rotated_eyes = calculate_rotated_eyes(rotated_image,eye_cascade,'normal')
    
    if len(rotated_eyes)!=2:
        print('Rotated Detection Failed : not human or invalid image')
        print('='*65)
        continue
        
    print('Rotated Detection success')
    print('='*65)
    
    current_mask_area = extract_facial_mask_area(rotated_image,rotated_eyes)
    mask_area.append(current_mask_area)
    cv2.imwrite('D:/downloads/gen_img/nose_mask_1000_extracted/'+file, current_mask_area) # replace it with your path

02240-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02241-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02242-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02243-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02244-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02245-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02246-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02247-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02248-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02249-with-nose-mask(nose_tip).png detection result
In

Initial Detection Failed : not human or invalid image
02289-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02290-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02291-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02292-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02293-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02294-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02295-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02296-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02297-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02

Initial Detection success
Rotated Detection success
02339-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02340-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02341-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02342-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02343-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02344-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02345-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02346-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02347-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
0234

Initial Detection success
Rotated Detection Failed : not human or invalid image
02388-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02389-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection Failed : not human or invalid image
02390-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02391-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02392-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02393-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02394-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02395-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02396-with-nose-mask(nose_tip).png detection result


Initial Detection Failed : not human or invalid image
02437-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02438-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02439-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
02440-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02441-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02442-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02443-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02444-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
02445-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection succes

...

Initial Detection Failed : not human or invalid image
03023-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03024-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03025-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03026-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03027-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03028-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03029-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03030-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03031-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image


Initial Detection success
Rotated Detection Failed : not human or invalid image
03071-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03072-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03073-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03074-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03075-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03076-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03077-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03078-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03079-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rota

Initial Detection Failed : not human or invalid image
03120-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03121-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03122-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03123-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03124-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03125-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03126-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03127-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection Failed : not human or invalid image
03128-with-nose-mask(nose_tip).png detection result
Initial Detection succes

03167-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03168-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03169-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03170-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03171-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03172-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03173-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03174-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03175-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03176-with-nose-mask(nose_tip).png detection result
In

Initial Detection Failed : not human or invalid image
03216-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03217-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03218-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03219-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03220-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03221-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03222-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03223-with-nose-mask(nose_tip).png detection result
Initial Detection success
Rotated Detection success
03224-with-nose-mask(nose_tip).png detection result
Initial Detection Failed : not human or invalid image
03

In [30]:
print(len(mask_area))

439
